# Weather Data Parser
- Gets the WEATHER data into a dataframe ready for analytics
- Does not perform any analysis or non-reversible mods

TODO: 

In [35]:
# Preliminary setup
import pandas as pd
import zipfile
import os
import numpy as np

zipname = 'wx'
dataname = 'LFPO'
#nskiprows = int(1.1e5)
nskiprows = 0
#readrows = int(1.e5)
readrows = None # Comment out for smaller test load.

In [2]:
# Extract zip if necessary
if not os.path.isfile(dataname + '.txt'):
    zip_ref = zipfile.ZipFile('src/' + zipname +'.zip', 'r')
    zip_ref.extractall('.')
    zip_ref.close()

In [3]:
# Setup for read_csv
coltypes = { 'alti': 'float64', 'drct': 'float64', 'dwpf': 'float64', 'gust': 'float64', 'metar': 'str', 'mslp': 'float64', 'p01i': 'float64', 'relh': 'float64', 'sknt': 'float64', 'skyc1': 'object', 'skyc2': 'object', 'skyc3': 'object', 'skyc4': 'object', 'skyl1': 'float64', 'skyl2': 'float64', 'skyl3': 'float64', 'skyl4': 'float64', 'station': 'object', 'tmpf': 'float64', 'valid': 'object', 'vsby': 'float64', 'wxcodes': 'object'}
# This dict should work to only allow M as null in the numeric columns... but it doesn't.
#null_m_cols = {'alti': 'M', 'drct': 'M', 'dwpf': 'M', 'gust': 'M', 'mslp': 'M', 'p01i': 'M', 'relh': 'M', 'sknt': 'M', 'skyl1': ['M'], 'skyl2': 'M', 'skyl3': 'M', 'skyl4': 'M', 'tmpf': 'M', 'vsby': 'M'}

In [81]:
df = pd.read_csv(dataname + '.txt'
                 #,nrows=3000
                 ,low_memory=False
                 ,dtype=coltypes
                 ,parse_dates=['valid']
                 ,na_values='M'
                )
df.rename(columns=lambda x: x.strip(),inplace=True)
#df.dtypes.to_dict()
df.index = df.valid
df.drop(['station', 'valid', 'sknt', 'p01i', 'mslp'
    ,'vsby', 'gust', 'skyc1', 'skyc2', 'skyc3', 'skyc4'
    ,'skyl1', 'skyl2', 'skyl3', 'skyl4', 'wxcodes', 'metar'
        ],axis=1,inplace=True)

In [79]:
# Store it for later cleanup
df.to_pickle("allwxdata.pickle")

In [80]:
df

,tmpf,dwpf,relh,drct,alti
valid,,,,,
2007-01-01 06:30:00,50.0,42.8,76.18,240.0,30.12
2007-01-01 07:00:00,50.0,42.8,76.18,230.0,30.12
2007-01-01 07:30:00,50.0,42.8,76.18,230.0,30.15
2007-01-01 08:00:00,50.0,42.8,76.18,230.0,30.15
2007-01-01 08:30:00,46.4,41.0,81.34,250.0,30.15
2007-01-01 09:00:00,48.2,41.0,76.01,250.0,30.18
2007-01-01 09:30:00,48.2,39.2,70.88,260.0,30.21
2007-01-01 10:00:00,48.2,41.0,76.01,250.0,30.21
2007-01-01 10:30:00,50.0,39.2,66.27,260.0,30.24
